In [1]:
import classes
import time
import datetime
import pandas as pd
from tqdm import tqdm


#open import data as pandas dataframe
patient_data = pd.read_csv("./data/PACJENT.TXT",sep=";",header=0)
test_data = pd.read_csv("./data/BADANIA2.TXT",sep=";",header=0)
bacteria_data = pd.read_csv("./data/DROBN.TXT",sep=";",header=0)
drag_sus_data = pd.read_csv("./data/LEKOWRAZ.TXT",sep=";",header=0)

In [38]:
print("Number of unfiltered patient records: " +  str(len(patient_data)))
print("Number of unfiltered tests records: " +  str(len(test_data)))
print("Number of unfiltered bacteria records: " +  str(len(bacteria_data)))
print("Number of unfiltered sus records: " +  str(len(drag_sus_data)))

Number of unfiltered patient records: 6861
Number of unfiltered tests records: 17198
Number of unfiltered bacteria records: 15553
Number of unfiltered sus records: 136286


In [3]:
test_classes = []

for index, data in test_data.iterrows():
    try:
        temp = classes.Test(data["badanie id"], data["id pobytu"], data["data"], data["kod materialu"])
        if temp.is_date_between( datetime.date(1992,1,1), datetime.date(1996,3,31)):
            test_classes.append(temp)
        else:
            pass
            #print("date outside of range at line: "+ str( index) )
            #print("date: " + str(temp.date) + "\n")
    except ValueError as e:
        print(str(e))
        print("bad value at line " + str(index) )
        continue 

year 199200 is out of range
bad value at line 1472
invalid literal for int() with base 10: '1993:00:00'
bad value at line 5849
year 19930 is out of range
bad value at line 5880
year 19940 is out of range
bad value at line 8777
year 19940 is out of range
bad value at line 10233
invalid literal for int() with base 10: '19950:00:00'
bad value at line 14552
invalid literal for int() with base 10: '19950:00:00'
bad value at line 14583


In [4]:
delta = len(test_data) - len(test_classes)
print("new test's lenght = {}, delta = {}".format(len(test_classes), delta))
    

new test's lenght = 16995, delta = 203


In [5]:
patient_classes = []
for index, data in patient_data.iterrows():
    temp_class = classes.Patient(data["numer"], data["sex"], data["depName"])
    if temp_class.PatientId not in [klass.PatientId for klass in patient_classes]:
        patient_classes.append(temp_class)
    else:
        print("same id at index: " + str(index))
        for i in range(len(patient_classes)):
            if temp_class.PatientId == patient_classes[i].PatientId:
                print("with id at: " + str(i))
                if temp_class.department[0] not in patient_classes[i].department:
                    patient_classes[i].extend_department(temp_class)
                    print(patient_classes[i].department)
                    break
print(len(patient_classes))

same id at index: 726
with id at: 725
['oiom', 'neur']
same id at index: 2225
with id at: 2223
['WEWN', 'nacz']
same id at index: 3326
with id at: 3323
['nefr', 'szcz']
same id at index: 3327
with id at: 3323
same id at index: 3670
with id at: 3665
['PLAS', 'urol']
same id at index: 3872
with id at: 3866
['WEWN', 'nacz']
same id at index: 3873
with id at: 3866
same id at index: 4185
with id at: 4177
['REUM', 'nefr']
same id at index: 4916
with id at: 4907
['NEFR', 'nech']
6852


In [6]:
print("table lenght {}".format(len(patient_data)))
print("classes array table len " + str( patient_classes.__len__()) )
print("dwa powtórzenia i 7 złączeń")

table lenght 6861
classes array table len 6852
dwa powtórzenia i 7 złączeń


In [7]:
bacteria_classes = [classes.Bacteria(data["drobnoustroj id"],
                                           data["id badania"],
                                           data["kod drobnoustroju"])
                                                        for index, data in bacteria_data.iterrows()]
drug_classes = [classes.DrugSusceptibility(data["id drobnoustroju"],
                                           data["kod leku"],
                                           data["opornosc"])
                                                        for index, data in drag_sus_data.iterrows()]

In [8]:
table_of_test_id = [klass.testId for klass in test_classes]
table_of_patient_id = [klass.PatientId for klass in patient_classes]

In [9]:
to_remove = []
for i in range(len(test_classes)):
    if test_classes[i].PatientId not in table_of_patient_id:
        to_remove.append(test_classes[i])
        
lenght_before = len(test_classes)
for o in to_remove:
    test_classes.remove(o)
len_after = len(test_classes)
print("badania filtrowanie: długość przed {} długość po {} delta {} ".format(lenght_before,len_after,lenght_before-len_after))
    
to_remove = []
for i in range(len(bacteria_classes)):
    if bacteria_classes[i].testId not in table_of_test_id:
        to_remove.append(bacteria_classes[i])
lenght_before = len(bacteria_classes)

for o in to_remove:
    bacteria_classes.remove(o)
    
len_after = len(bacteria_classes)
print("bakteria filtrowanie: długość przed {} długość po {} delta {} ".format(lenght_before,len_after,lenght_before-len_after))

badania filtrowanie: długość przed 16995 długość po 16995 delta 0 
bakteria filtrowanie: długość przed 15553 długość po 15421 delta 132 


In [10]:
to_remove = []

table_of_bacteria_id = [klass.bacteriaId for klass in bacteria_classes]
for i in range(len(drug_classes)):
    if drug_classes[i].DrugId not in table_of_bacteria_id:
        to_remove.append(drug_classes[i])
        
lenght_before = len(drug_classes)        

for o in to_remove:
    try:
        drug_classes.remove(o)    
    except ValueError as e:
        print(str(e))
len_after = len(drug_classes)
print("drug filtrowanie: długość przed {} długość po {} delta {} ".format(lenght_before,len_after,lenght_before-len_after))

drug filtrowanie: długość przed 136286 długość po 135220 delta 1066 


In [ ]:
#budowanie drzewa zależności od antybiotyku w górę
for drug in tqdm(drug_classes):
    for i in range(len(bacteria_classes)):
        if drug.DrugId == bacteria_classes[i].bacteriaId:
            bacteria_classes[i].add_susceptibility(drug)
            break

In [12]:
bez_badania = 0
z_badaniem = 0
for bacteria in bacteria_classes:
    if bacteria.DrugSusceptibility:
        z_badaniem+=1
    else:
        bez_badania+=1
print("z badaniem {} bez {}".format(z_badaniem,bez_badania))

z badaniem 13505 bez 1916


In [13]:
#bakterie do badania - help :(
for bakteria in tqdm(bacteria_classes):
    for i in range(len(test_classes)):
        if bakteria.testId == test_classes[i].testId:
            test_classes[i].add_bacteria(bakteria)
            break

100%|██████████| 15421/15421 [01:12<00:00, 212.01it/s]


In [14]:
bez_bakteri = 0
z_bakterio = 0
for test in test_classes:
    if test.Bacteria:
        z_bakterio+=1
    else:
        bez_bakteri+=1
print("z bakterią {} bez {}".format(z_badaniem,bez_badania))

z badaniem 13505 bez 1916


In [27]:
#badania do pacjętów - there is no god he left us all
for test in tqdm(test_classes):
    for i in range(len(patient_classes)):
        if test.PatientId == patient_classes[i].PatientId:
            patient_classes[i].add_test(test)
            break

100%|██████████| 16995/16995 [00:07<00:00, 2285.53it/s]


In [29]:
bez_badania= 0
z_badaniem = 0
for patient in patient_classes:
    if patient.test:
        z_badaniem+=1
    else:
        bez_badania+=1
print("z badaniem {} bez {}".format(z_badaniem,bez_badania))

z badaniem 6746 bez 106


In [46]:
temp = [klass for klass in  patient_classes if klass.test and klass.test[0].materialId == 120]
#120 pochwa i z tego wychodzą same 0 czli 0-kob 1-mężcz

0
0
0
0
0
0
0
